# Connect

In [10]:
# Panda hostname/IP and Desk login information of your robot
hostname = '172.16.0.2'
username = 'TQSH'
password = 'tqshanghai'
base_dir = '../../data/hand_in_eye8'
# panda-py is chatty, activate information log level
import logging
import panda_py

logging.basicConfig(level=logging.INFO)

In [11]:
from panda_py import libfranka

panda = panda_py.Panda(hostname)
panda.enable_logging(int(1e2))
gripper = libfranka.Gripper(hostname)

INFO:panda:Connected to robot (172.16.0.2).
INFO:panda:Panda class destructor invoked (172.16.0.2).
INFO:panda:Stopping active controller (Trajectory).


# Hand in eye calib

In [12]:
from panda_py import constants
from scipy.spatial.transform import Rotation as R
import numpy as np
import matplotlib.pyplot as plt
import os
import time

In [13]:
def generate_and_move_to_pose(init_pose, roll, pitch, yaw, z_add, x_add, y_add, max_roll_deviation, max_pitch_deviation, max_yaw_deviation):
    """Generate a new pose with turbulence and move the robot arm to it."""
    roll_turbulent = roll + np.random.uniform(-max_roll_deviation, max_roll_deviation)
    pitch_turbulent = pitch + np.random.uniform(-max_pitch_deviation, max_pitch_deviation)
    yaw_turbulent = yaw + np.random.uniform(-max_yaw_deviation, max_yaw_deviation)

    r = R.from_euler('xyz', [roll_turbulent, pitch_turbulent, yaw_turbulent], degrees=False)
    rotation_matrix = r.as_matrix()

    absolute_rotation_matrix = np.dot(init_pose[:3, :3], rotation_matrix)

    pose = init_pose.copy()
    pose[:3, :3] = absolute_rotation_matrix
    pose[2, 3] += z_add
    pose[0, 3] += x_add
    pose[1, 3] += y_add

    panda.move_to_pose(pose)
    
    return pose

def save_pose(pose, base_dir, frame_num):
    """Save the robot arm's pose to a file."""
    pose_filename = f'{base_dir}/poses/pose_{frame_num}.npy'
    np.save(pose_filename, pose)
    print(f"Saved pose to {pose_filename}")

def save_joints(joints, base_dir, frame_num):
    """Save the robot arm's joints to a file."""
    joints_filename = f'{base_dir}/joints/joints_{frame_num}.npy'
    np.save(joints_filename, joints)
    print(f"Saved joints to {joints_filename}")


In [14]:
import sys
parent_dir = os.path.dirname(os.getcwd())
parent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
from realsense.realsense import Camera
from realsense.realsense import get_devices


def capture_images(camera, delay_before_shooting, start_frame, picture_nums, base_dir, init_pose, roll, pitch, yaw, z_add, x_add, y_add,
                   max_roll_deviation, max_pitch_deviation, max_yaw_deviation):
    
    camera.start()
    
    rgb_intrinsics, rgb_coeffs, depth_intrinsics, depth_coeffs = camera.get_intrinsics_raw()
    depth_scale = camera.get_depth_scale()

    print(f"RGB Intrinsics: {rgb_intrinsics}")
    print(f"RGB Distortion Coefficients: {rgb_coeffs}")
    rgb_intrinsics_path = f'{base_dir}/rgb_intrinsics.npz'
    np.savez(rgb_intrinsics_path, fx=rgb_intrinsics.fx, fy=rgb_intrinsics.fy, ppx=rgb_intrinsics.ppx, ppy=rgb_intrinsics.ppy, coeffs=rgb_intrinsics.coeffs)

    print(f"Depth Scale: {depth_scale}")
    print(f"Depth Intrinsics: {depth_intrinsics}")
    print(f"Depth Distortion Coefficients: {depth_coeffs}")
    depth_intrinsics_path = f'{base_dir}/depth_intrinsics.npz'
    np.savez(depth_intrinsics_path, fx=depth_intrinsics.fx, fy=depth_intrinsics.fy, ppx=depth_intrinsics.ppx, ppy=depth_intrinsics.ppy, coeffs=depth_intrinsics.coeffs, depth_scale=depth_scale)

    # drop the first few frames to allow the camera to warm up
    _, _ = camera.shoot()  
    time.sleep(delay_before_shooting)

    for frame_num in range(start_frame, start_frame + picture_nums):  # Capture images at 10 different poses
        pose = generate_and_move_to_pose(init_pose, roll, pitch, yaw, z_add, x_add, y_add,
                                         max_roll_deviation, max_pitch_deviation, max_yaw_deviation)
        rgb_image, depth_image = camera.shoot()
        rgb_filename = f'{base_dir}/rgb/{frame_num}.png'
        depth_filename = f'{base_dir}/depth/{frame_num}.npy'
        plt.imsave(rgb_filename, rgb_image)
        np.save(depth_filename, depth_image)
        print(f"Saved {rgb_filename}")
        print(f"Saved {depth_filename}")

        save_pose(pose, base_dir, frame_num)
        last_qpos = panda.get_log()["q"][-1]
        save_joints(last_qpos, base_dir, frame_num)

    panda.move_to_start()
        
    camera.stop()


In [15]:
os.makedirs(f'{base_dir}/rgb', exist_ok=True)
os.makedirs(f'{base_dir}/depth', exist_ok=True)
os.makedirs(f'{base_dir}/poses', exist_ok=True)
os.makedirs(f'{base_dir}/joints', exist_ok=True)
# Define a list of configurations
image_configs = [
    {
        'base_dir': '../hand_in_eye2',
        'init_pose': panda_py.fk(constants.JOINT_POSITION_START),
        'roll': 0.0,
        'pitch': 0.2,
        'yaw': 0.0,
        'z_add': 0.16,
        'x_add': 0.12,
        'y_add': 0.0,
        'max_roll_deviation': 0.1,
        'max_pitch_deviation': 0.1,
        'max_yaw_deviation': 0.1
    },
    {
        'base_dir': '../hand_in_eye2',
        'init_pose': panda_py.fk(constants.JOINT_POSITION_START),
        'roll': 0.3,
        'pitch': 0.35,
        'yaw': 0.0,
        'z_add': 0.1,
        'x_add': 0.0,
        'y_add': -0.25,
        'max_roll_deviation': 0.1,
        'max_pitch_deviation': 0.1,
        'max_yaw_deviation': 0.1
    },
    {
        'base_dir': '../hand_in_eye2',
        'init_pose': panda_py.fk(constants.JOINT_POSITION_START),
        'roll': -0.25,
        'pitch': 0.35,
        'yaw': 0.2,
        'z_add': 0.12,
        'x_add': 0.1,
        'y_add': 0.20,
        'max_roll_deviation': 0.1,
        'max_pitch_deviation': 0.1,
        'max_yaw_deviation': 0.1
    },
    {
        'base_dir': '../hand_in_eye2',
        'init_pose': panda_py.fk(constants.JOINT_POSITION_START),
        'roll': -0.25,
        'pitch': 0.2,
        'yaw': 0.2,
        'z_add': 0.20,
        'x_add': 0.11,
        'y_add': 0.1,
        'max_roll_deviation': 0.1,
        'max_pitch_deviation': 0.1,
        'max_yaw_deviation': 0.1
    }
]


# Enumerate connected RealSense cameras
device_serials = get_devices()

# Print selected device serial numbers
print("Selected device serial numbers:", device_serials[0])

rgb_resolution = (640, 480)  # RGB resolution (width, height)
depth_resolution = (640, 480)  # Depth resolution (width, height)

camera = Camera(device_serials[0], rgb_resolution, depth_resolution)

# Delay before shooting (in seconds)
delay_before_shooting = 3.0

# Iterate over the list of configurations and capture images
for i, config in enumerate(image_configs):
    capture_images(camera, delay_before_shooting, 10*i, 10, base_dir, config['init_pose'], config['roll'], config['pitch'], config['yaw'],
                   config['z_add'], config['x_add'], config['y_add'], config['max_roll_deviation'],
                   config['max_pitch_deviation'], config['max_yaw_deviation'])

Selected device serial numbers: 115222071207
RGB Intrinsics: [ 640x480  p[330.552 241.445]  f[604.902 605.103]  Inverse Brown Conrady [0 0 0 0 0] ]
RGB Distortion Coefficients: [0.0, 0.0, 0.0, 0.0, 0.0]
Depth Scale: 0.0010000000474974513
Depth Intrinsics: [ 640x480  p[322.025 240.057]  f[379.037 379.037]  Brown Conrady [0 0 0 0 0] ]
Depth Distortion Coefficients: [0.0, 0.0, 0.0, 0.0, 0.0]


INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.60 seconds.
INFO:panda:Starting new controller (Trajectory).
ERROR:panda:Control loop interruped: libfranka: Move command rejected: command not possible in the current mode ("User stopped")!
INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.50 seconds.
INFO:panda:Starting new controller (Trajectory).
ERROR:panda:Control loop interruped: libfranka: Move command rejected: command not possible in the current mode ("User stopped")!
INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.66 seconds.
INFO:panda:Starting new controller (Trajectory).
ERROR:panda:Control loop interruped: libfranka: Move command rejected: command not poss

Saved ../../hand_in_eye8/rgb/0.png
Saved ../../hand_in_eye8/depth/0.npy
Saved pose to ../../hand_in_eye8/poses/pose_0.npy
Saved joints to ../../hand_in_eye8/joints/joints_0.npy
Saved ../../hand_in_eye8/rgb/1.png
Saved ../../hand_in_eye8/depth/1.npy
Saved pose to ../../hand_in_eye8/poses/pose_1.npy
Saved joints to ../../hand_in_eye8/joints/joints_1.npy
Saved ../../hand_in_eye8/rgb/2.png
Saved ../../hand_in_eye8/depth/2.npy
Saved pose to ../../hand_in_eye8/poses/pose_2.npy
Saved joints to ../../hand_in_eye8/joints/joints_2.npy


INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.59 seconds.
INFO:panda:Starting new controller (Trajectory).
ERROR:panda:Control loop interruped: libfranka: Move command rejected: command not possible in the current mode ("User stopped")!
INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.51 seconds.
INFO:panda:Starting new controller (Trajectory).
ERROR:panda:Control loop interruped: libfranka: Move command rejected: command not possible in the current mode ("User stopped")!
INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.50 seconds.
INFO:panda:Starting new controller (Trajectory).
ERROR:panda:Control loop interruped

Saved ../../hand_in_eye8/rgb/3.png
Saved ../../hand_in_eye8/depth/3.npy
Saved pose to ../../hand_in_eye8/poses/pose_3.npy
Saved joints to ../../hand_in_eye8/joints/joints_3.npy
Saved ../../hand_in_eye8/rgb/4.png
Saved ../../hand_in_eye8/depth/4.npy
Saved pose to ../../hand_in_eye8/poses/pose_4.npy
Saved joints to ../../hand_in_eye8/joints/joints_4.npy
Saved ../../hand_in_eye8/rgb/5.png
Saved ../../hand_in_eye8/depth/5.npy
Saved pose to ../../hand_in_eye8/poses/pose_5.npy
Saved joints to ../../hand_in_eye8/joints/joints_5.npy


...

Saved ../../hand_in_eye8/rgb/37.png
Saved ../../hand_in_eye8/depth/37.npy
Saved pose to ../../hand_in_eye8/poses/pose_37.npy
Saved joints to ../../hand_in_eye8/joints/joints_37.npy


INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.47 seconds.
INFO:panda:Starting new controller (Trajectory).


Saved ../../hand_in_eye8/rgb/38.png
Saved ../../hand_in_eye8/depth/38.npy
Saved pose to ../../hand_in_eye8/poses/pose_38.npy
Saved joints to ../../hand_in_eye8/joints/joints_38.npy


INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 2.67 seconds.
INFO:panda:Starting new controller (Trajectory).


Saved ../../hand_in_eye8/rgb/39.png
Saved ../../hand_in_eye8/depth/39.npy
Saved pose to ../../hand_in_eye8/poses/pose_39.npy
Saved joints to ../../hand_in_eye8/joints/joints_39.npy
